In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# The N-Queens-Problem as a CSP

The function `create_csp(n)` takes a natural number `n` as argument and returns
a *constraint satisfaction problem* that encodes the 
[`n`-queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle).
    
A constraint satisfaction problem $\mathcal{P}$ is a triple of the form
$$ \mathcal{P} = \langle \mathtt{Vars}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
where 
- `Vars` is a set of strings which serve as *variables*.

  The idea is that $V_i$ specifies the column of the queen that is placed in row $i$.
    
- `Values` is a set of *values* that can be assigned 
  to the variables in $\mathtt{Vars}$.
  
  In the 8-queens-problem we will have $\texttt{Values} = \{1,\cdots,8\}$.
- `Constraints` is a set of formulas from first order logic.  
  Each of these formulas is  called a *constraint* of $\mathcal{P}$.
  There are two different types of constraints.
  * We have constraints that express that no two queens that are positioned in different rows share the same
    column.  To capture these constraints, we define
    $$\texttt{DifferentCol} := \bigl\{ \texttt{V}_i \not= \texttt{V}_j \bigm| i \in \{1,\cdots,8\} \wedge j \in \{1,\cdots,8\} \wedge j < i \bigr\}.$$
    Here the condition $j < i$ ensures that, for example,  while we have the constraint
    $\texttt{V}_2 \not= \texttt{V}_1$ we do not also have the constraint  $\texttt{V}_1 \not= \texttt{V}_2$, as the latter 
    constraint would be redundant if the former constraint had already been established.
  * We have constraints that express that no two queens positioned in different rows share the same 
    diagonal.  The queens in row $i$ and row $j$ share the same diagonal iff the equation
    $$ |i - j| = |V_i - V_j| $$
    holds.  The expression $|i-j|$ is the absolute value of the difference of the rows of the queens in row
    $i$ and row $j$,  while the expression $|V_i - V_j|$ is the absolute value of the difference of the
    columns of these queens.  To capture these constraints, we define
    $$ \texttt{DifferentDiag} := \bigl\{ |i  - j| \not= |\texttt{V}_i - \texttt{V}_j| \bigm| i \in \{1,\cdots,8\} \wedge j \in \{1,\cdots,8\} \wedge j < i \bigr\}. $$

In [ ]:
def create_csp(n):
    S              = range(1, n+1)           
    Variables      = { f'V{i}' for i in S }
    Values         = set(S)
    DifferentCols  = { f'V{i} != V{j}' for i in S
                                       for j in S
                                       if  i < j 
                     }
    DifferentDiags = { f'abs(V{j} - V{i}) != {j - i}' for i in S
                                                      for j in S 
                                                      if  i < j 
                     }
    return Variables, Values, DifferentCols | DifferentDiags

The function `main()` creates a CSP representing the 4-queens puzzle and prints the CSP.
It is included for testing purposes.

In [ ]:
def main():
    Vars, Values, Constraints = create_csp(4)
    print('Variables:  ', Vars)
    print('Values:     ', Values)
    print('Constraints:')
    for c in Constraints:
        print('            ', c)

In [ ]:
main()

## Displaying the Solution

The following functions have been contributed by Phillip Polland with minor edits from me.

We use the [ElementTree XML API](https://docs.python.org/3.10/library/xml.etree.elementtree.html) below and have to import it.

In [ ]:
import xml.etree.ElementTree as ET

Furthermore, we use the module [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/). They can be installed in the commandline with the following command:
```
    conda install -c conda-forge ipywidge
```

In [ ]:
from ipywidgets import Layout, HTML

In [ ]:
def _svg(viewbox: int, size: int = None) -> ET.Element:
    svg = ET.Element("svg", {
        "xmlns": "http://www.w3.org/2000/svg",
        "xmlns:xlink": "http://www.w3.org/1999/xlink",
        "version": "1.2",
        "baseProfile": "tiny",
        "viewBox": f"0 0 {viewbox:d} {viewbox:d}",
    })
    if size is not None:
        svg.set("width", str(size))
        svg.set("height", str(size))
    return svg

The function `paint_nxn_board(n, Qs)` paints a chessboard with queens.
* `n` specifies the number of fields of the board.  For a classical chessboard, `n` is eight.
* `Qs` specifies the positions of the queens. `Qs[col]` specifies that in column `col` the queen
  is placed in row `Qs[col]`.

In [ ]:
def paint_nxn_board(n=8, Qs=None, SQUARE_SIZE=16, size=None):
    svg    = _svg(n * SQUARE_SIZE, size)
    fields = [(file, rank) for file in range(n) for rank in range(n)]
    for file_index, rank_index in fields:
        ET.SubElement(svg, "rect", {
            "x"      : str(file_index * SQUARE_SIZE),
            "y"      : str(rank_index * SQUARE_SIZE),
            "width"  : str(SQUARE_SIZE),
            "height" : str(SQUARE_SIZE),
            "fill"   : "#ffce9e" if (file_index + rank_index) % 2 == 0 else "#d18b47"
        })
    if Qs:
        with open('queen.svg') as f:
            QUEEN_SVG_STRING = f.read()
        defs = ET.SubElement(svg, "defs")
        defs.append(ET.fromstring(QUEEN_SVG_STRING))
        scale = SQUARE_SIZE / 45
        if any(q >= n or q < 0 for q in Qs): 
            raise ValueError("At least one queen is in an impossible field")
        for file_index, rank_index in enumerate(Qs):
            x = file_index * SQUARE_SIZE
            y = rank_index * SQUARE_SIZE
            href = f"#white-queen"
            ET.SubElement(svg, "use", {"href": href, "transform": f"translate({x:d}, {y:d}) scale({scale:f})"})
    return HTML(ET.tostring(svg).decode("utf-8"))

The function `show_solution(Solution)` takes a dictionary that contains a variable assignment that represents a solution to the 8-queens puzzle.  It displays this Solution on a chess board.

In [ ]:
def show_solution(Solution, size=500):
    n      = len(Solution)
    Queens = [None for col in range(n)]
    for row in range(n):
        col = Solution['V'+str(row+1)]
        Queens[col-1] = row
    return paint_nxn_board(n, Queens, SQUARE_SIZE=16, size=size)

In [ ]:
Solution = {'V5': 1, 'V2': 2, 'V4': 6, 'V6': 3, 'V1': 7, 'V7': 5, 'V3': 4}

In [ ]:
show_solution(Solution)

In [ ]:
Solution2 = {'V5': 1, 'V2': 2, 'V4': 6, 'V6': 3, 'V1': 7, 'V7': 5, 'V3': 4, 'V9':8, 'V8': 9}

In [ ]:
show_solution(Solution2, 700)